<a href="https://colab.research.google.com/github/praseedm/Learn-Deep-Learning/blob/main/Pytorch_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Src : https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
torch.__version__

'2.0.0+cu118'

# QUICK START

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [6]:
training_data =  datasets.CIFAR10(
    root="data",
    train=True, download=True,
    transform=ToTensor()
)

100%|██████████| 170498071/170498071 [00:11<00:00, 14998285.79it/s]


Extracting data/cifar-10-python.tar.gz to data


In [7]:
type(training_data)

torchvision.datasets.cifar.CIFAR10

In [8]:
test_data =  datasets.CIFAR10(
    root="data",
    train=False, download=True,
    transform=ToTensor()
)

Files already downloaded and verified


In [9]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size) 

In [23]:
for X,y in test_dataloader:
  print(X.shape, X.dtype)
  print(y.shape, y.dtype)
  print(nn.Flatten()(X).shape)
  break

# Shape [BS, C, H, W] -> [64, 3, 32, 32]

torch.Size([64, 3, 32, 32]) torch.float32
torch.Size([64]) torch.int64
torch.Size([64, 3072])


In [20]:
device = ("cude" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device

'cpu'

In [30]:
from torch.nn.modules import activation
class SimpleNeuralNetwork(nn.Module):
  def __init__(self, *args, **kwargs) -> None:
    super().__init__(*args, **kwargs)
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(3*32*32, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits



In [31]:
simpleNN = SimpleNeuralNetwork().to(device)
print(simpleNN)

SimpleNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [32]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(simpleNN.parameters(), lr=1e-3)

In [33]:
X,y = None, None
for X,y in test_dataloader:
  break

In [36]:
logits = simpleNN(X)
logits.shape

torch.Size([64, 10])

In [37]:
logits[0]

tensor([ 0.0199, -0.0699,  0.0193,  0.0429, -0.0351,  0.0703,  0.0492, -0.0206,
         0.0592, -0.0030], grad_fn=<SelectBackward0>)

In [51]:
nn.Softmax()(logits)[0]

<ipython-input-51-3250ec9e0e54>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  nn.Softmax()(logits)[0]


tensor([0.1006, 0.0919, 0.1005, 0.1029, 0.0952, 0.1058, 0.1036, 0.0966, 0.1046,
        0.0983], grad_fn=<SelectBackward0>)

In [53]:
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X,y) in enumerate(dataloader):
    X,y = X.to(device), y.to(device)

    logits = model(X)
    loss = loss_fn(logits, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [54]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, simpleNN, loss_fn, optimizer)
    # test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.316805  [   64/50000]
loss: 2.302919  [ 6464/50000]
loss: 2.296076  [12864/50000]
loss: 2.290252  [19264/50000]
loss: 2.279833  [25664/50000]
loss: 2.280330  [32064/50000]
loss: 2.277413  [38464/50000]
loss: 2.283768  [44864/50000]
Epoch 2
-------------------------------
loss: 2.293254  [   64/50000]
loss: 2.284386  [ 6464/50000]
loss: 2.255482  [12864/50000]
loss: 2.271093  [19264/50000]
loss: 2.244455  [25664/50000]
loss: 2.250868  [32064/50000]
loss: 2.257148  [38464/50000]
loss: 2.248914  [44864/50000]
Epoch 3
-------------------------------
loss: 2.266219  [   64/50000]
loss: 2.253937  [ 6464/50000]
loss: 2.205043  [12864/50000]
loss: 2.244653  [19264/50000]
loss: 2.197256  [25664/50000]
loss: 2.207957  [32064/50000]
loss: 2.229769  [38464/50000]
loss: 2.195575  [44864/50000]
Epoch 4
-------------------------------
loss: 2.229261  [   64/50000]
loss: 2.210581  [ 6464/50000]
loss: 2.136760  [12864/50000]
loss: 2.207912  [19264/50000]
